In [7]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
spark = SparkSession.builder.appName("PySparkTables").getOrCreate()

# Sample data
employee_data = [
    (1, 'A', 100),
    (2, 'A', 200),
    (3, 'A', 300),
    (4, 'B', 400),
    (5, 'B', 500),
    (6, 'C', 100)
]

# Define schema
employee_schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("company", StringType(), False),
    StructField("salary", IntegerType(), False),
])

# Create DataFrame
employee_df = spark.createDataFrame(employee_data, employee_schema)

# Register as temporary view
employee_df.createOrReplaceTempView("Employee")


In [22]:
spark.sql("""
    with cte as (
    select *,
    row_number() over(partition by company order by salary desc) as rn
    from Employee),
     
    second_highest as (select company, salary
    from cte where rn = 2)
    
    select DISTINCT e.company, s.salary from Employee as e left join second_highest as s on e.company = s.company
""").show()

+-------+------+
|company|salary|
+-------+------+
|      A|   200|
|      B|   400|
|      C|  null|
+-------+------+



----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 55296)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/lib/python3.7/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/lib/python3.7/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/lib/python3.7/socketserver.py", line 720, in __init__
    self.handle()
  File "/spark/python/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/spark/python/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/spark/python/pyspark/accumulators.py", line 257, in accum_updates
    num_updates = read_int(self.rfile)
  File "/spark/python/pyspark/serializers.py", line 595, in read_i